Nome: Francisco Jerônimo da Silva Júnior <br>
Matrícula: 433399

In [60]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from mlxtend.plotting import plot_decision_regions

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron

In [61]:
import warnings
warnings.filterwarnings('ignore')

### Leitura dos dados

In [62]:
df = pd.read_csv('../Lista-07/diabetes.csv')

### Visão inicial dos dados

In [63]:
df.sample(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
69,4,146,85,27,100,28.9,0.189,27,0
353,1,90,62,12,43,27.2,0.580,24,0
232,1,79,80,25,37,25.4,0.583,22,0
701,6,125,78,31,0,27.6,0.565,49,1
407,0,101,62,0,0,21.9,0.336,25,0
478,8,126,74,38,75,25.9,0.162,39,0
281,10,129,76,28,122,35.9,0.280,39,0
185,7,194,68,28,0,35.9,0.745,41,1
367,0,101,64,17,0,21.0,0.252,21,0
399,3,193,70,31,0,34.9,0.241,25,1


### 1. Separe os dados em treino (80%) e teste (20%).

In [64]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

### 2. Crie os modelos a seguir sobre o conjunto de treino e calcule F1-Score sobre os dados de treino e de teste usando:

- Random Forest.
- Gradient Boosting do Scikit Learn
- XGBoost - https://xgboost.readthedocs.io/
- LightGBM - https://lightgbm.readthedocs.io/
- CatBoost - https://catboost.ai/
- Perceptron - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html
- Multi-layer Perceptron - https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

#### Função para treino, predição e avaliação


In [65]:
def model_app(model):
    # treinamento do modelo
    model.fit(X_train, y_train)
    
    # predição
    y_train_pred, y_test_pred = model.predict(X_train), model.predict(X_test) 
    
    # retorna o cálculo da métrica f1_score aplicada nos valores preditos com os dados
    # de treino e teste
    print(f'Dados de treino: {f1_score(y_train, y_train_pred)}')
    print(f'Dados de teste: {f1_score(y_test, y_test_pred)}')

#### Random Forest

In [66]:
rfi = RandomForestClassifier(n_estimators=500, n_jobs=-1)

In [67]:
model_app(rfi)

Dados de treino: 1.0
Dados de teste: 0.6095238095238096


#### Gradient Boosting

Valores baixos de learning rate requerem mais árvores no ensemble

In [68]:
gbc = GradientBoostingClassifier(max_depth=2, n_estimators=500, learning_rate=0.1)

In [69]:
model_app(gbc)

Dados de treino: 0.9569377990430622
Dados de teste: 0.6422018348623852


#### XGboost

In [70]:
xgbc = XGBClassifier(n_estimators=300)

In [71]:
model_app(xgbc)

[11:34:56] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Dados de treino: 1.0
Dados de teste: 0.6095238095238096


#### LightGBM 

In [72]:
lgbm = LGBMClassifier(n_estimators=300)

In [73]:
model_app(lgbm)

Dados de treino: 1.0
Dados de teste: 0.5599999999999999


#### CatBoost

In [74]:
cbc = CatBoostClassifier()

In [75]:
model_app(cbc)

Learning rate set to 0.008365
0:	learn: 0.6883809	total: 1.85ms	remaining: 1.85s
1:	learn: 0.6848605	total: 4.58ms	remaining: 2.29s
2:	learn: 0.6806187	total: 7.53ms	remaining: 2.5s
3:	learn: 0.6769906	total: 9.6ms	remaining: 2.39s
4:	learn: 0.6728245	total: 11.4ms	remaining: 2.27s
5:	learn: 0.6682649	total: 13.2ms	remaining: 2.18s
6:	learn: 0.6643114	total: 15.2ms	remaining: 2.15s
7:	learn: 0.6604106	total: 17.6ms	remaining: 2.18s
8:	learn: 0.6568639	total: 19.5ms	remaining: 2.15s
9:	learn: 0.6537936	total: 21.3ms	remaining: 2.1s
10:	learn: 0.6498616	total: 24.1ms	remaining: 2.17s
11:	learn: 0.6470807	total: 26.5ms	remaining: 2.18s
12:	learn: 0.6441885	total: 28.2ms	remaining: 2.14s
13:	learn: 0.6400866	total: 30.1ms	remaining: 2.12s
14:	learn: 0.6366405	total: 32.1ms	remaining: 2.11s
15:	learn: 0.6324074	total: 34.7ms	remaining: 2.13s
16:	learn: 0.6292021	total: 37.3ms	remaining: 2.16s
17:	learn: 0.6263366	total: 40ms	remaining: 2.18s
18:	learn: 0.6240038	total: 41.8ms	remaining: 2.1

172:	learn: 0.4111192	total: 557ms	remaining: 2.66s
173:	learn: 0.4106666	total: 560ms	remaining: 2.66s
174:	learn: 0.4099292	total: 562ms	remaining: 2.65s
175:	learn: 0.4093829	total: 564ms	remaining: 2.64s
176:	learn: 0.4087157	total: 566ms	remaining: 2.63s
177:	learn: 0.4080587	total: 571ms	remaining: 2.64s
178:	learn: 0.4074817	total: 576ms	remaining: 2.64s
179:	learn: 0.4067900	total: 578ms	remaining: 2.63s
180:	learn: 0.4062880	total: 580ms	remaining: 2.62s
181:	learn: 0.4056383	total: 582ms	remaining: 2.61s
182:	learn: 0.4044906	total: 584ms	remaining: 2.6s
183:	learn: 0.4036392	total: 587ms	remaining: 2.6s
184:	learn: 0.4029549	total: 589ms	remaining: 2.59s
185:	learn: 0.4024815	total: 592ms	remaining: 2.59s
186:	learn: 0.4016700	total: 594ms	remaining: 2.58s
187:	learn: 0.4009995	total: 597ms	remaining: 2.58s
188:	learn: 0.4002821	total: 604ms	remaining: 2.59s
189:	learn: 0.3998255	total: 609ms	remaining: 2.6s
190:	learn: 0.3990700	total: 614ms	remaining: 2.6s
191:	learn: 0.39

336:	learn: 0.3424700	total: 1.13s	remaining: 2.22s
337:	learn: 0.3421237	total: 1.13s	remaining: 2.22s
338:	learn: 0.3418921	total: 1.14s	remaining: 2.21s
339:	learn: 0.3415482	total: 1.14s	remaining: 2.21s
340:	learn: 0.3413248	total: 1.14s	remaining: 2.2s
341:	learn: 0.3411018	total: 1.14s	remaining: 2.19s
342:	learn: 0.3409544	total: 1.14s	remaining: 2.19s
343:	learn: 0.3406584	total: 1.15s	remaining: 2.18s
344:	learn: 0.3403918	total: 1.15s	remaining: 2.18s
345:	learn: 0.3402379	total: 1.15s	remaining: 2.17s
346:	learn: 0.3397908	total: 1.15s	remaining: 2.17s
347:	learn: 0.3395246	total: 1.15s	remaining: 2.16s
348:	learn: 0.3392552	total: 1.15s	remaining: 2.15s
349:	learn: 0.3387303	total: 1.16s	remaining: 2.15s
350:	learn: 0.3384612	total: 1.16s	remaining: 2.14s
351:	learn: 0.3383362	total: 1.16s	remaining: 2.14s
352:	learn: 0.3381724	total: 1.16s	remaining: 2.13s
353:	learn: 0.3378011	total: 1.16s	remaining: 2.13s
354:	learn: 0.3375484	total: 1.17s	remaining: 2.12s
355:	learn: 0

525:	learn: 0.2952869	total: 1.7s	remaining: 1.53s
526:	learn: 0.2951021	total: 1.7s	remaining: 1.52s
527:	learn: 0.2948260	total: 1.7s	remaining: 1.52s
528:	learn: 0.2946558	total: 1.7s	remaining: 1.52s
529:	learn: 0.2944622	total: 1.71s	remaining: 1.51s
530:	learn: 0.2942377	total: 1.71s	remaining: 1.51s
531:	learn: 0.2940756	total: 1.71s	remaining: 1.5s
532:	learn: 0.2939675	total: 1.71s	remaining: 1.5s
533:	learn: 0.2937770	total: 1.71s	remaining: 1.49s
534:	learn: 0.2935965	total: 1.71s	remaining: 1.49s
535:	learn: 0.2933393	total: 1.72s	remaining: 1.49s
536:	learn: 0.2930281	total: 1.72s	remaining: 1.48s
537:	learn: 0.2929126	total: 1.72s	remaining: 1.48s
538:	learn: 0.2927742	total: 1.72s	remaining: 1.47s
539:	learn: 0.2925190	total: 1.72s	remaining: 1.47s
540:	learn: 0.2924496	total: 1.72s	remaining: 1.46s
541:	learn: 0.2922836	total: 1.73s	remaining: 1.46s
542:	learn: 0.2921036	total: 1.73s	remaining: 1.45s
543:	learn: 0.2919458	total: 1.73s	remaining: 1.45s
544:	learn: 0.2917

747:	learn: 0.2532092	total: 2.07s	remaining: 699ms
748:	learn: 0.2531024	total: 2.08s	remaining: 696ms
749:	learn: 0.2529494	total: 2.08s	remaining: 693ms
750:	learn: 0.2528950	total: 2.08s	remaining: 690ms
751:	learn: 0.2527214	total: 2.08s	remaining: 686ms
752:	learn: 0.2525565	total: 2.08s	remaining: 683ms
753:	learn: 0.2522704	total: 2.08s	remaining: 680ms
754:	learn: 0.2520405	total: 2.09s	remaining: 677ms
755:	learn: 0.2519488	total: 2.09s	remaining: 674ms
756:	learn: 0.2518653	total: 2.09s	remaining: 671ms
757:	learn: 0.2516824	total: 2.09s	remaining: 668ms
758:	learn: 0.2513208	total: 2.09s	remaining: 665ms
759:	learn: 0.2511457	total: 2.1s	remaining: 662ms
760:	learn: 0.2509476	total: 2.1s	remaining: 659ms
761:	learn: 0.2507446	total: 2.1s	remaining: 655ms
762:	learn: 0.2504846	total: 2.1s	remaining: 653ms
763:	learn: 0.2503437	total: 2.1s	remaining: 649ms
764:	learn: 0.2502181	total: 2.1s	remaining: 646ms
765:	learn: 0.2499780	total: 2.1s	remaining: 643ms
766:	learn: 0.24979

946:	learn: 0.2178091	total: 2.46s	remaining: 138ms
947:	learn: 0.2176053	total: 2.46s	remaining: 135ms
948:	learn: 0.2173873	total: 2.46s	remaining: 132ms
949:	learn: 0.2171857	total: 2.47s	remaining: 130ms
950:	learn: 0.2170116	total: 2.47s	remaining: 127ms
951:	learn: 0.2168552	total: 2.47s	remaining: 125ms
952:	learn: 0.2167667	total: 2.47s	remaining: 122ms
953:	learn: 0.2165116	total: 2.47s	remaining: 119ms
954:	learn: 0.2163926	total: 2.47s	remaining: 117ms
955:	learn: 0.2162275	total: 2.48s	remaining: 114ms
956:	learn: 0.2160886	total: 2.48s	remaining: 111ms
957:	learn: 0.2159232	total: 2.48s	remaining: 109ms
958:	learn: 0.2157448	total: 2.48s	remaining: 106ms
959:	learn: 0.2156996	total: 2.48s	remaining: 103ms
960:	learn: 0.2155494	total: 2.48s	remaining: 101ms
961:	learn: 0.2155245	total: 2.49s	remaining: 98.2ms
962:	learn: 0.2153742	total: 2.49s	remaining: 95.6ms
963:	learn: 0.2152619	total: 2.49s	remaining: 93ms
964:	learn: 0.2151495	total: 2.49s	remaining: 90.4ms
965:	learn

#### Perceptron

In [76]:
pc = Perceptron()

In [77]:
model_app(pc)

Dados de treino: 0.0928270042194093
Dados de teste: 0.1515151515151515


#### Multi-layer Perceptron 

In [78]:
# definição dos modelos
mlpc1 = MLPClassifier(activation='tanh')
mlpc2 = MLPClassifier(activation='relu')
mlpc3 = MLPClassifier(activation='identity')

In [91]:
for model in [mlpc1, mlpc2, mlpc3]:
    print(f'Função de ativação: {model.activation}')
    model_app(model)
    print()

Função de ativação: tanh
Dados de treino: 0.6682134570765662
Dados de teste: 0.5892857142857143

Função de ativação: relu
Dados de treino: 0.3208955223880597
Dados de teste: 0.3333333333333333

Função de ativação: identity
Dados de treino: 0.5645933014354066
Dados de teste: 0.5576923076923077

